In [4]:

"""OTE Client."""
from otelib.strategies import DataResource, Filter, Function, Mapping, Transformation


class OTEClient:
    """The OTEClient object representing a remote OTE REST API.

    Parameters:
        url (str): The base URL of the OTEAPI Service.

    Attributes:
        url (str): The base URL of the OTEAPI Service.

    """

    def __init__(self, url: str) -> None:
        """Initiates an OTEAPI Service client.

        The `url` is the base URL of the OTEAPI Service.
        """
        self.url: str = url

    def create_dataresource(self, **kwargs) -> DataResource:
        """Create a new data resource.

        Any given keyword arguments are passed on to the `create()` method.

        Returns:
            The newly created data resource.

        """
        data_resource = DataResource(self.url)
        data_resource.create(**kwargs)
        return data_resource

    def create_transformation(self, **kwargs) -> Transformation:
        """Create a new transformation.

        Any given keyword arguments are passed on to the `create()` method.

        Returns:
            The newly created transformation.

        """
        transformation = Transformation(self.url)
        transformation.create(**kwargs)
        return transformation

    def create_filter(self, **kwargs) -> Filter:
        """Create a new filter.

        Any given keyword arguments are passed on to the `create()` method.

        Returns:
            The newly created filter.

        """
        filter_ = Filter(self.url)
        filter_.create(**kwargs)
        return filter_

    def create_mapping(self, **kwargs) -> Mapping:
        """Create a new mapping.

        Any given keyword arguments are passed on to the `create()` method.

        Returns:
            The newly created mapping.

        """
        mapping = Mapping(self.url)
        mapping.create(**kwargs)
        return mapping

    def create_function(self, **kwargs) -> Function:
        """Create a new function.

        Any given keyword arguments are passed on to the `create()` method.

        Returns:
            The newly created function.

        """
        function_ = Function(self.url)
        function_.create(**kwargs)
        return function_


In [10]:
"""Abstract Base Class (abc) for strategies."""
import json
import os
from abc import ABC, abstractmethod
from typing import TYPE_CHECKING

import requests

from otelib.exceptions import ApiError
from otelib.pipe import Pipe
from otelib.settings import Settings
from uuid import uuid4

if TYPE_CHECKING:  # pragma: no cover
    from pathlib import Path
    from typing import Optional

class AbstractPythonStrategy(ABC):
    """Abstract class for strategies for the python backend.

    Parameters:
        url (str): The base URL of the OTEAPI Service.

    Attributes:
        url (str): The base URL of the OTEAPI Service.
        settings (otelib.settings.Settings): OTEAPI Service settings.
        input_pipe (Optional[Pipe]): An input pipeline.

    """

    def __init__(
        self, url: "Optional[str]" = None, py_exec: "Optional[Path]" = None
    ) -> None:
        """Initiates a strategy."""
        if not url and not py_exec or all((url, py_exec)):
            raise ValueError("Either url or py_exec must be specified, not both.")

        self.url: str = url
        self.settings: Settings = Settings()
        self.input_pipe: "Optional[Pipe]" = None
        self.id: "Optional[str]" = None  # pylint: disable=invalid-name

        # For debugging/testing
        self.debug: bool = bool(os.getenv("OTELIB_DEBUG", ""))
        self._session_id: "Optional[str]" = None

    @abstractmethod
    def create(self, **kwargs) -> None:
        """Create a strategy.

        It should post the configuration for the created strategy.
        """

    @abstractmethod
    def fetch(self, session_id: str) -> bytes:
        """Returns the result of the current strategy.

        This method is called by `get()` while propagating up the pipeline.

        Parameters:
            session_id: The ID of the session shared by the pipeline.

        Returns:
            The response from the OTEAPI Service.

        """

    @abstractmethod
    def initialize(self, session_id: str) -> bytes:
        """Initialise the current strategy.

        This method is called by `get()` when propagating down the pipeline.

        Parameters:
            session_id: The ID of the session shared by the pipeline.

        Returns:
            The response from the OTEAPI Service.

        """

    def _set_input(self, input_pipe: Pipe) -> None:
        """Used by `__rshift__` to set the input pipe.

        Parameters:
            input_pipe: A pipe representing the strategy that is "piped" into this
                strategy.

        """
        self.input_pipe = input_pipe

    def __rshift__(self, other: "AbstractStrategy") -> "AbstractStrategy":
        """Implements strategy concatenation using the `>>` symbol.

        Parameters:
            other: The next strategy this one is "piping" into.

        Returns:
            The next strategy this one is "piped" into.

        """
        pipe = Pipe(self)
        other._set_input(pipe)
        return other

    def get(self, session_id: "Optional[str]" = None) -> bytes:
        """Executes a pipeline.

        This will call `initialize()` and then the `get()` method on the
        input pipe, which in turn will call the `get()` method on the
        strategy connected to its input and so forth until the beginning
        of the pipeline.

        Finally, `fetch()` is called and its output is returned.

        Parameters:
            session_id: The ID of the session shared by the pipeline.

        Returns:
            The output from `fetch()`.

        """
        self.settings = Settings()

        if session_id is None:
            session_id = 'session-'+str(uuid4())

        if self.debug:
            self._session_id = session_id

        self.initialize(session_id)
        if self.input_pipe:
            self.input_pipe.get(session_id)
        return self.fetch(session_id)


In [28]:

"""Common strategy for Download, Prase and Resource strategies."""
import requests
from oteapi.models import ResourceConfig
from uuid import uuid4

from otelib.exceptions import ApiError
from otelib.strategies.abc import AbstractStrategy

cache = {} # Can turn into a proper object later...

class DataResource(AbstractPythonStrategy):
    """Context class for the data resource strategy interfaces for managing i/o
    operations."""

    def create(self, **kwargs) -> None:
        session_id = kwargs.pop("session_id", None)
        data = ResourceConfig(**kwargs)

        resource_id = 'dataresource-'+str(uuid4())
        cache[resource_id] = data.json()

        if session_id:
            session = cache[session_id]
            list_key = "resource_info"
            if list_key in session:
                session[list_key].extend([resource_id])
            else:
                session[list_key] = [resource_id]

        return 

    def fetch(self, session_id: str) -> bytes:
        response = requests.get(
            f"{self.url}{self.settings.prefix}/dataresource/{self.id}",
            params={"session_id": session_id},
        )
        if response.ok:
            return response.content
        raise ApiError(
            f"Cannot fetch data resource: session_id={session_id!r} "
            f"resource_id={self.id!r}"
            f"{' content=' + str(response.content) if self.debug else ''}",
            status=response.status_code,
        )

    def initialize(self, session_id: str) -> bytes:
        config = ResourceConfig(cache[])


In [31]:
# taken from OTEClient create_dataresource


kwargs = {
    "downloadUrl":"https://jpeg.org/images/jpegsystems-home.jpg",
    "mediaType":"image/jpeg",
}
data_resource = DataResource('python')
data_resource.create(**kwargs)

In [27]:
cache

{'dataresource-ef645a5c-8deb-4fe7-a06f-35adcd2e3f1b': '{"configuration": {}, "description": "Resource Strategy Data Configuration.\\n\\n    Important:\\n        Either of the pairs of attributes `downloadUrl`/`mediaType` or\\n        `accessUrl`/`accessService` MUST be specified.\\n\\n    ", "downloadUrl": "https://jpeg.org/images/jpegsystems-home.jpg", "mediaType": "image/jpeg", "accessUrl": null, "accessService": null, "license": null, "accessRights": null, "publisher": null}'}